In [ ]:
!pip install transformers==4.5

     |████████████████████████████████| 2.2MB 29.9MB/s 
     |████████████████████████████████| 901kB 40.7MB/s 
     |████████████████████████████████| 3.3MB 27.2MB/s 


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 30.8MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 7.8MB 41.0MB/s 
     |████████████████████████████████| 133kB 52.9MB/s 
     |████████████████████████████████| 1.8MB 45.2MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 46.8MB/s 
     |████████████████████████████████| 266kB 46.0MB/s 
     |████████████████████████████████| 4.2MB 44.2MB/s 
     |████████████████████████████████| 174kB 57.2MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 112kB 48.5MB/s 
     |████████████████████████████████| 143kB 51.2MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 245kB 43.7MB/s 
     |████████████████████████████████| 122kB 52.8MB/s 
     |████████████████████████████████| 122kB 57.2MB/s 
     |████████████████████████████████| 71kB 11.2MB/

In [ ]:
!pip install torch==1.9.0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, BertModel, BertTokenizer, BertForSequenceClassification, BertTokenizerFast
from scipy.special import softmax
from datetime import datetime
from sklearn.utils import resample

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('drive/MyDrive/train_quotes.csv')
test = pd.read_csv('drive/MyDrive/test_quotes.csv')

In [ ]:
train = train.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Author'])
test = test.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Author'])

In [ ]:
train['top25pct'] = (train['Likes'] >= 89).astype(int)
test['top25pct'] = (test['Likes'] >= 89).astype(int)

In [ ]:
def upsample_minority(df):

  # Upsample minority class in both the training and test data
  df_majority = df.loc[df['top25pct'] == 0, :]
  df_minority = df.loc[df['top25pct'] == 1, :]
  df_minority_upsampled = resample(df_minority, replace = True, n_samples = len(df_majority), random_state = 42)

  # Combine together to get the upsampled training data
  df = pd.concat([df_majority, df_minority_upsampled])

  return df

In [ ]:
# Upsample the minority class
train = upsample_minority(train)
test = upsample_minority(test)

train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [ ]:
from simpletransformers.classification import ClassificationModel
import logging
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train['Quote'], train['top25pct'], test_size = 0.2, random_state = 20, stratify = train['top25pct'])

train_data = pd.concat([X_train, y_train], axis = 1)
valid_data = pd.concat([X_valid, y_valid], axis = 1)
test_data = pd.concat([test['Quote'], test['top25pct']], axis = 1)

# Rename columns
train_data.columns = ['text', 'labels']
valid_data.columns = ['text', 'labels']
test_data.columns = ['text', 'labels']

In [ ]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

68193
17049
21298


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

args = {
   'output_dir': 'drive/MyDrive/quote_bert_model/',
   'cache_dir': 'cache/',

   'fp16': False,

   'overwrite_output_dir': True,
   'reprocess_input_data': True,
}

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', args = args)

INFO:filelock:Lock 140330921193808 acquired on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307.lock


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

INFO:filelock:Lock 140330921193808 released on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307.lock
INFO:filelock:Lock 140330785643984 acquired on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

INFO:filelock:Lock 140330785643984 released on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expec

...

INFO:filelock:Lock 140330783319696 released on /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock
INFO:filelock:Lock 140330785488400 acquired on /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6.lock


Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

INFO:filelock:Lock 140330785488400 released on /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6.lock
INFO:filelock:Lock 140330785506896 acquired on /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140330785506896 released on /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f.lock


In [ ]:
train_data.head()

,text,labels
71075,"You were the missing piece of my soul, the bre...",1
5960,"Unfortunately, most people need to know a lot ...",0
55038,What are you doing with all those books anyway...,1
82302,The lack of money is the root of all evil.,1
66956,"That's how you know you love someone, I guess,...",1


In [ ]:
# Train the model
model.train_model(train_data, output_dir='drive/MyDrive/', eval_df = valid_data, args = {'num_train_epochs': 6})

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/68193 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache/cached_train_bert_128_2_2


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/8525 [00:00<?, ?it/s]

...

Running Epoch 3 of 6:   0%|          | 0/8525 [00:00<?, ?it/s]

Running Epoch 4 of 6:   0%|          | 0/8525 [00:00<?, ?it/s]

Running Epoch 5 of 6:   0%|          | 0/8525 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to drive/MyDrive/.


(51150, 0.25424278678223683)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_data, acc = accuracy_score)
test_probs = softmax(model_outputs, axis = 1)
test_preds = np.argmax(test_probs, axis = 1)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/21298 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/2663 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.2644682113852303, 'tp': 4553, 'tn': 8691, 'fp': 1958, 'fn': 6096, 'auroc': 0.6831477472623945, 'auprc': 0.6698248437275196, 'acc': 0.621842426518922, 'eval_loss': 2.620559479324675}


In [ ]:
confusion_matrix(test_data['labels'], test_preds)

array([[8691, 1958],
       [6096, 4553]])